In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_numopt
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

In [2]:
device = "cpu"

In [3]:
class Net(nn.Module):
    def __init__(self, input_size, device="cpu"):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)

        return x

In [4]:
# X, y = load_diabetes(return_X_y=True, scaled=False)
# X, y = make_regression(n_samples=1000, n_features=100)
X, y = make_friedman1(n_samples=1000, noise=1e-2)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)
X = torch.Tensor(X).to(device)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))
y = torch.Tensor(y).to(device)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

torch_data = TensorDataset(X_train, y_train)
data_loader = DataLoader(torch_data, batch_size=1000)

torch.Size([800, 10]) torch.Size([800, 1])
torch.Size([200, 10]) torch.Size([200, 1])


In [5]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = torch_numopt.AGD(
    model=model,
    lr=1,
    mu=0.001,
    mu_dec=0.1,
    use_diagonal=False,
    c1=1e-4,
    tau=0.5,
    line_search_method="backtrack",
    line_search_cond="armijo",
)

all_loss = []
patience = 0
max_patience = 10
for epoch in range(100):
    print("epoch: ", epoch, end="")
    all_loss.append(0)
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch] += loss

    all_loss[epoch] /= len(data_loader)

    if epoch > 0 and all_loss[epoch - 1] <= all_loss[epoch]:
        patience -= 1
    else:
        patience = max_patience

    print(", loss: {}".format(all_loss[epoch].detach().cpu().numpy().item()))

    if patience <= 0:
        break

epoch:  0, loss: 0.4928751289844513
epoch:  1, loss: 0.2277892678976059
epoch:  2, loss: 0.20452290773391724
epoch:  3, loss: 0.11910425126552582
epoch:  4, loss: 0.014592969790101051
epoch:  5, loss: 0.010401159524917603
epoch:  6, loss: 0.010214787907898426
epoch:  7, loss: 0.00910700298845768
epoch:  8, loss: 0.008887479081749916
epoch:  9, loss: 0.00857711210846901
epoch:  10, loss: 0.008355841971933842
epoch:  11, loss: 0.008006341755390167
epoch:  12, loss: 0.0076352995820343494
epoch:  13, loss: 0.006370640359818935
epoch:  14, loss: 0.006293326150625944
epoch:  15, loss: 0.005120198242366314
epoch:  16, loss: 0.004983052145689726
epoch:  17, loss: 0.004961550701409578
epoch:  18, loss: 0.004648225381970406
epoch:  19, loss: 0.00454372214153409
epoch:  20, loss: 0.0044871787540614605
epoch:  21, loss: 0.004363866522908211
epoch:  22, loss: 0.004291203338652849
epoch:  23, loss: 0.0041812751442193985
epoch:  24, loss: 0.004168940242379904
epoch:  25, loss: 0.004072027280926704
ep

In [6]:
pred_train = model.forward(X_train).detach()
pred_test = model.forward(X_test).detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train)}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test)}")

Train metrics: R2 = 0.9869390729385881
Test metrics:  R2 = 0.9858332090313062


In [7]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = torch_numopt.AGD(
    model=model,
    lr=1,
    mu=0.001,
    mu_dec=0.1,
    use_diagonal=True,
    c1=1e-4,
    tau=0.1,
    line_search_method="backtrack",
    line_search_cond="armijo",
)

all_loss = []
patience = 0
max_patience = 10
for epoch in range(100):
    print("epoch: ", epoch, end="")
    all_loss.append(0)
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch] += loss

    all_loss[epoch] /= len(data_loader)

    if epoch > 0 and all_loss[epoch - 1] <= all_loss[epoch]:
        patience -= 1
    else:
        patience = max_patience

    print(", loss: {}".format(all_loss[epoch].detach().cpu().numpy().item()))

    if patience <= 0:
        break

epoch:  0, loss: 0.473205029964447
epoch:  1, loss: 0.3820722699165344
epoch:  2, loss: 0.3008015751838684
epoch:  3, loss: 0.24184522032737732
epoch:  4, loss: 0.11892692744731903
epoch:  5, loss: 0.013040613383054733
epoch:  6, loss: 0.010139682330191135
epoch:  7, loss: 0.008021051995456219
epoch:  8, loss: 0.006697891745716333
epoch:  9, loss: 0.005750979296863079
epoch:  10, loss: 0.005151944234967232
epoch:  11, loss: 0.004576586652547121
epoch:  12, loss: 0.004169571213424206
epoch:  13, loss: 0.0038120809476822615
epoch:  14, loss: 0.003420913126319647
epoch:  15, loss: 0.0030864491127431393
epoch:  16, loss: 0.002792555605992675
epoch:  17, loss: 0.0025406316854059696
epoch:  18, loss: 0.0023104448337107897
epoch:  19, loss: 0.002110644243657589
epoch:  20, loss: 0.0019379639998078346
epoch:  21, loss: 0.0017896337667480111
epoch:  22, loss: 0.0016643169801682234
epoch:  23, loss: 0.0015239000786095858
epoch:  24, loss: 0.0012564589269459248
epoch:  25, loss: 0.001144718378782

In [8]:
pred_train = model.forward(X_train).detach()
pred_test = model.forward(X_test).detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train)}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test)}")

Train metrics: R2 = 0.9910935767870599
Test metrics:  R2 = 0.9884574572398859
